In [1]:
# Import dependencies
import pandas as pd
import requests
import datetime as dt
import os

In [3]:
# Read in the winter csv
winter_df = pd.read_csv("../Outputs/winter_df.csv", dtype={'start_station_id':"string", "end_station_id":"string"})

# Preview the df
winter_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,weather_join
0,7B06EEA21608591A,classic_bike,2022-11-12 14:47:21,2022-11-12 14:58:55,Newport PATH,JC066,7 St & Monroe St,HB304,40.727146,-74.033620,40.746413,-74.037977,casual,2022-11-12 14:00:00
1,B22A3E8765F1E165,classic_bike,2022-11-30 08:29:53,2022-11-30 08:45:56,Newport PATH,JC066,7 St & Monroe St,HB304,40.727216,-74.033609,40.746413,-74.037977,member,2022-11-30 08:00:00
2,CFB2EB1663C02BB6,electric_bike,2022-11-09 08:28:30,2022-11-09 08:44:50,Newport PATH,JC066,7 St & Monroe St,HB304,40.727224,-74.033759,40.746413,-74.037977,member,2022-11-09 08:00:00
3,85700816DAAE1C36,electric_bike,2022-11-30 14:48:11,2022-11-30 14:58:10,Newport PATH,JC066,7 St & Monroe St,HB304,40.727224,-74.033759,40.746413,-74.037977,member,2022-11-30 14:00:00
4,009EBB1F54C5947E,electric_bike,2022-11-10 18:14:33,2022-11-10 18:23:08,Newport PATH,JC066,7 St & Monroe St,HB304,40.727224,-74.033759,40.746413,-74.037977,member,2022-11-10 18:00:00


In [4]:
# Check the column dtypes
winter_df.dtypes

ride_id                       object
rideable_type                 object
started_at                    object
ended_at                      object
start_station_name            object
start_station_id      string[python]
end_station_name              object
end_station_id        string[python]
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
weather_join                  object
dtype: object

In [5]:
# Obtain the min datetime from the 'started_at' column
min_datetime = pd.to_datetime(winter_df['started_at']).dt.date.min()

# Obtain the max datetime from the 'started_at' column
max_datetime = pd.to_datetime(winter_df['started_at']).dt.date.max()

# Calculate the number of hour periods will be required
periods = (max_datetime - min_datetime).days * 24
print(periods)

2184


In [6]:
# Create an hour frequency between the min and max dates
frequency = pd.date_range(min_datetime, periods=periods,freq="H")
frequency

DatetimeIndex(['2022-11-01 00:00:00', '2022-11-01 01:00:00',
               '2022-11-01 02:00:00', '2022-11-01 03:00:00',
               '2022-11-01 04:00:00', '2022-11-01 05:00:00',
               '2022-11-01 06:00:00', '2022-11-01 07:00:00',
               '2022-11-01 08:00:00', '2022-11-01 09:00:00',
               ...
               '2023-01-30 14:00:00', '2023-01-30 15:00:00',
               '2023-01-30 16:00:00', '2023-01-30 17:00:00',
               '2023-01-30 18:00:00', '2023-01-30 19:00:00',
               '2023-01-30 20:00:00', '2023-01-30 21:00:00',
               '2023-01-30 22:00:00', '2023-01-30 23:00:00'],
              dtype='datetime64[ns]', length=2184, freq='H')

In [7]:
# Obtain hourly weather data from Open-Meteo
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.71,
	"longitude": -74.06,
	"start_date": min_datetime,
	"end_date": max_datetime,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "weather_code", "wind_speed_10m", "is_day", "sunshine_duration"],
    "wind_speed_unit":"mph",
	"timezone": "auto"
}
responses = requests.get(url, params=params)

In [8]:
# Read the response as a json format
response = responses.json()

# View the keys of the json file
response['hourly'].keys()

dict_keys(['time', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain', 'snowfall', 'weather_code', 'wind_speed_10m', 'is_day', 'sunshine_duration'])

In [9]:
# Process hourly data. The order of variables needs to be the same as requested.
# Code adjusted from the autogenerated on open-meteo.com documentation

## 'print(len())' commands were added to ensure the correct number of values were returned

hourly = response['hourly']
hourly_temperature_2m = hourly['temperature_2m']
# print(len(hourly_temperature_2m))
hourly_relative_humidity_2m = hourly['relative_humidity_2m']
# print(len(hourly_relative_humidity_2m))
hourly_precipitation = hourly['precipitation']
# print(len(hourly_precipitation))
hourly_rain = hourly['rain']
# print(len(hourly_rain))
hourly_snowfall = hourly['snowfall']
# print(len(hourly_snowfall))
hourly_weather_code = hourly['weather_code']
# print(len(hourly_weather_code))
hourly_wind_speed_10m = hourly['wind_speed_10m']
# print(len(hourly_wind_speed_10m))
hourly_is_day = hourly['is_day']
# print(len(hourly_is_day))
hourly_sunshine_duration = hourly['sunshine_duration']
# print(len(hourly_sunshine_duration))

hourly_data = {"date": pd.date_range(
	start = min_datetime,
	end = max_datetime+dt.timedelta(days=1),
	freq = "H",
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["weather_code"] = hourly_weather_code
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

# print(len(hourly_data["date"]))

# Put the data into a dataframe
hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,wind_speed_10m,is_day,sunshine_duration
0,2022-11-01 00:00:00,14.2,97,0.7,0.7,0.0,53,4.5,0,0.0
1,2022-11-01 01:00:00,14.4,97,0.3,0.3,0.0,51,4.9,0,0.0
2,2022-11-01 02:00:00,14.3,98,1.7,1.7,0.0,61,2.6,0,0.0
3,2022-11-01 03:00:00,13.9,98,2.6,2.6,0.0,63,5.5,0,0.0
4,2022-11-01 04:00:00,14.0,98,0.8,0.8,0.0,53,3.9,0,0.0


In [10]:
# View the 'weather_code' values
hourly_dataframe['weather_code'].value_counts()

weather_code
0     933
1     413
3     358
2     163
51    146
53     65
61     56
63     30
55     21
71     11
73      8
65      3
75      1
Name: count, dtype: int64

In [11]:
# View the count of day/night values
# 1 = Day / 0 = Night
hourly_dataframe['is_day'].value_counts()

is_day
0    1341
1     867
Name: count, dtype: int64

In [12]:
# Produce a set of the 'weather_code' to remove duplicates
weather_code_set = set(hourly_dataframe['weather_code'])

# View the codes
weather_code_set

{0, 1, 2, 3, 51, 53, 55, 61, 63, 65, 71, 73, 75}

In [13]:
# Check the df info
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  2208 non-null   datetime64[ns]
 1   temperature_2m        2208 non-null   float64       
 2   relative_humidity_2m  2208 non-null   int64         
 3   precipitation         2208 non-null   float64       
 4   rain                  2208 non-null   float64       
 5   snowfall              2208 non-null   float64       
 6   weather_code          2208 non-null   int64         
 7   wind_speed_10m        2208 non-null   float64       
 8   is_day                2208 non-null   int64         
 9   sunshine_duration     2208 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3)
memory usage: 172.6 KB


In [14]:
# Convert the 'weather_code' and 'is_day' columns to strings
# This is to prepare for mapping values

hourly_dataframe[['weather_code', 'is_day']] = hourly_dataframe[['weather_code','is_day']].astype('string', errors="raise")

In [15]:
# create map dicts for weather code and day/night

weathercodes = {}

for i in weather_code_set:
    weathercodes[str(i)] = ""

day_night = {
    "0":"Night",
    "1":"Day"
}

In [16]:
# Weather codes obtained from:
# https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c
weathercodes = {
    '0': 'Clear',
    '1': 'Mainly Clear',
    '2': 'Partly Cloudy',
    '3': 'Cloudy',
    '65': 'Heavy Rain',
    '71': 'Light Snow',
    '73': 'Snow',
    '75': 'Heavy Snow',
    '51': 'Light Drizzle',
    '53': 'Drizzle',
    '55': 'Heavy Drizzle',
    '61': 'Light Rain',
    '63': 'Rain'}

In [17]:
# Map the new values to the dataframe
hourly_dataframe['is_day'] = hourly_dataframe['is_day'].map(day_night, na_action="ignore")
hourly_dataframe['weather_code'] = hourly_dataframe['weather_code'].map(weathercodes, na_action="ignore")

# Preview the df
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,wind_speed_10m,is_day,sunshine_duration
0,2022-11-01 00:00:00,14.2,97,0.7,0.7,0.0,Drizzle,4.5,Night,0.0
1,2022-11-01 01:00:00,14.4,97,0.3,0.3,0.0,Light Drizzle,4.9,Night,0.0
2,2022-11-01 02:00:00,14.3,98,1.7,1.7,0.0,Light Rain,2.6,Night,0.0
3,2022-11-01 03:00:00,13.9,98,2.6,2.6,0.0,Rain,5.5,Night,0.0
4,2022-11-01 04:00:00,14.0,98,0.8,0.8,0.0,Drizzle,3.9,Night,0.0


In [18]:
# Output the df to a new csv file
hourly_dataframe.to_csv("../Outputs/winter_weather.csv", index=False)